In [1]:
# Importing Necessary Libraries
import cdsapi
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
from sklearn.metrics import mean_squared_error
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import MSELoss
from torch.nn.init import xavier_uniform_

In [1]:
# CDS API Client Call
c = cdsapi.Client()

#Retrieve  Data for 2m Temperature from 1st December 2019 to 7th December 2019
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'variable': '2m_temperature',
        'year': '2019',
        'month': '12',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07',
        ],
        'time': '12:00',
    },
    '2m_temperature.nc')

#Retrieve  Data for Total Precipitation from 1st December 2019 to 7th December 2019
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'variable': 'total_precipitation',
        'year': '2019',
        'month': '12',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07',
        ],
        'time': '12:00',
    },
    'total_precipitation.nc')

#Retrieve  Data for Volumetric Soil Water Layer 1 from 1st December 2019 to 7th December 2019
c.retrieve(
    'reanalysis-era5-land',
    {
        'format': 'netcdf',
        'variable': 'volumetric_soil_water_layer_1',
        'year': '2019',
        'month': '12',
        'day': [
            '01', '02', '03',
            '04', '05', '06',
            '07',
        ],
        'time': '12:00',
    },
    'volumetric_soil_water_layer_1.nc')

In [1]:
#Open and describe 2m Temperature
A = xr.open_dataset("2m_temperature.nc")
A

In [1]:
#Open and describe Volumetric Soil Water Layer 1
B = xr.open_dataset("volumetric_soil_water_layer_1.nc")
B

In [1]:
#Open and describe Total Precipitation
C = xr.open_dataset("total_precipitation.nc")
C

In [1]:
#Merge 2m Temperature, Volumetric Soil Water Layer 1 & Total Precipitation into one
#Using common dimensions latitude, longitude & time
D = xr.merge([A, B, C], compat="override")
D

In [1]:
#Mean
D.mean(skipna = "true")
#Median
D.median(skipna = "true")
#Standard Deviation
D.std(skipna = "true")
#Variance
D.var(skipna = "true")
#From standard deviation and variance it was clear that the spread of 2m temperature was well across
#the whole range and did not need any processing. While the spread of volumetric soil water layer data
#was not as well spread but was well enough for any processing needs.
#The total precipitation on other hand was left skewed and needed normalisation such as boxcox normalisation.
#Check for NULL/NaN values
D.isnull()

In [1]:
#Dropping NaN values as they are for co-ordinates that refer to the oceanic waters
#and are not required for last mile prediction of land climate.
A = A.dropna(dim = "latitude", how = "all")
B = B.dropna(dim = "latitude", how = "all")
C = C.dropna(dim = "latitude", how = "all")
D = D.dropna(dim = "latitude", how = "all")

In [1]:
#2m Temperature for 3-12-2019
D.t2m.sel(time="2019-12-03").plot()
#Spread of 2m Temperature throughout the dataset
D.t2m.plot()

In [1]:
#Volumetric Soil Water Layer 1 for 3-12-2019
D.swvl1.sel(time="2019-12-03").plot()
#Spread of Volumetric Soil Water layer 1 throughout the dataset
D.swvl1.plot()

In [1]:
#Total Precipitation for 3-12-2019
D.tp.sel(time="2019-12-03").plot()
#Spread of total precipitation throughout the dataset
D.tp.plot()

In [1]:
#Interpolating data to reduce the gap in latitude and longitude from 0.1 to 0.05
#Thus quadrupling the dataset and increasing the prediction at last mile
#Creating New lognitude values at 0.05 difference
new_lon = np.linspace(A.longitude[0], A.longitude[-1], A.dims["longitude"] * 2)
#Creating New latitude values at 0.05 difference
new_lat = np.linspace(A.latitude[0], A.latitude[-1], A.dims["latitude"] * 2)
#Interpolating 2m teperature dataset for new latitude and longitude values
AI = A.interp(latitude=new_lat, longitude=new_lon)

In [1]:
new_lon = np.linspace(B.longitude[0], B.longitude[-1], B.dims["longitude"] * 2)
new_lat = np.linspace(B.latitude[0], B.latitude[-1], B.dims["latitude"] * 2)
#Interpolating volumetric soil wat dataset for new latitude and longitude values
BI = B.interp(latitude=new_lat, longitude=new_lon)

In [1]:
new_lon = np.linspace(C.longitude[0], C.longitude[-1], C.dims["longitude"] * 2)
new_lat = np.linspace(C.latitude[0], C.latitude[-1], C.dims["latitude"] * 2)
CI = C.interp(latitude=new_lat, longitude=new_lon)

In [1]:
#Creating new merged and interpolated dataset
DI = xr.merge([AI, BI, CI], compat="override")
DI.to_netcdf("Interpolated_Data.nc")

In [2]:
# dataset definition
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, path):
        # load the file as a dataframe
        df = xr.open_dataset(path).to_dataframe()
        df = df.dropna()
        # store the inputs and outputs
        self.X = df.values[:, -4:-1].astype('float32')
        self.y = df.values[:, -1].astype('float32')
        # ensure target has the right shape
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train, validate and test rows
    def get_splits(self, n_test=0.1, n_val=0.1):
        # determine sizes
        test_size = round(n_test * len(self.X))
        val_size = round(n_val * len(self.X))
        train_size = len(self.X) - test_size - val_size
        # calculate the split
        return random_split(self, [train_size, val_size, test_size])

In [3]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        xavier_uniform_(self.hidden1.weight)
        self.act1 = Sigmoid()
        # second hidden layer
        self.hidden2 = Linear(10, 8)
        xavier_uniform_(self.hidden2.weight)
        self.act2 = Sigmoid()
        # third hidden layer and output
        self.hidden3 = Linear(8, 1)
        xavier_uniform_(self.hidden3.weight)
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        return X

In [4]:
# prepare the dataset
def prepare_data(path):
    # load the dataset
    dataset = CSVDataset(path)
    # calculate split
    train, validate, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=False)
    val_dl = DataLoader(validate, batch_size=32, shuffle=False)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, val_dl, test_dl

In [5]:
# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = MSELoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(100):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()

In [6]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate mse
    mse = mean_squared_error(actuals, predictions)
    return mse

In [7]:
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [ ]:
# prepare the data
path = "Interpolated_Data.nc"
train_dl, val_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(val_dl.dataset), len(test_dl.dataset))
# define the network
model = MLP(2)
# train the model
train_model(train_dl, model)
# evaluate the model
mse = evaluate_model(test_dl, model)
print('MSE: %.3f, RMSE: %.3f' % (mse, sqrt(mse)))
# make a single prediction (expect class=1)
#row = [0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98]
#yhat = predict(row, model)
#print('Predicted: %.3f' % yhat)

46942823 5867853 5867853
